In [7]:
from prompts import PROMPT_ASSUNTO, PROMPT_DIFICULDADE, PROMPT_SUBASSUNTO, REGRAS_SAIDA
import json
import os
import pandas as pd
from src.codes.questao import Questao
from groq import Groq
import re
import time

Como deve ser chamado cada um dos prompts

```python
assunto = PROMPT_ASSUNTO.format(
	enunciado="Teste", 
	regras_da_saida=REGRAS_SAIDA.format(
		regra="Fundamentos da Matemática, Fundamentos de Computação ou Algoritmos e Estruturas de Dados"
	)
)

dificuldade = PROMPT_DIFICULDADE.format(
	enunciado="Teste", 
	regras_da_saida=REGRAS_SAIDA.format(
		regra="Fácil, Médio ou Difícil"
	)
)
subassunto = PROMPT_SUBASSUNTO.format(
	enunciado="Teste", 
	conteudo_estruturado_subassuntos = subassuntos('Algoritmos e Estruturas de Dados'),
	regras_da_saida=REGRAS_SAIDA.format(
		regra="<Grupo> \ <Subassunto> \ <Nível>"
	)
)
```

In [2]:
with open('src/data/conteudo.json', 'r') as file:
	conteudo = json.load(file)

In [3]:
def subassuntos(assunto: str) -> str:
	"""
	Retorna os subassuntos disponíveis para o assunto especificado.

	Args:
		assunto (str): O assunto para o qual se deseja obter os subassuntos.

	Returns:
		str: Uma string formatada contendo os subassuntos e seus tópicos correspondentes.
	"""
	indice = -1
	if assunto == 'Fundamentos de Matemática':
		indice = 0
	elif assunto == 'Fundamentos de Computação':
		indice = 1
	elif assunto == 'Algoritmos e Estruturas de Dados':
		indice = 2
	
	subassunto = ""
	for i,topico in enumerate(conteudo['conteudosEstruturados'][indice]['subsecoes'], start=1):
		subassunto += f"{i}. {topico['titulo']}\n"
		for subtopico in topico['topicos']:
			subassunto += f"\t[{subtopico['nivel']}] {subtopico['descricao']}\n"

	return subassunto

In [4]:
link_questoes = pd.read_csv('questoes.csv')

In [5]:
client = Groq(
	api_key=os.environ.get('API_KEY_GROQ'),
)

In [12]:
def obter_conteudo_formatado(prompt: str, model: str = 'llama-3.3-70b-versatile') -> str:
    resposta = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=model
    )
    conteudo = resposta.choices[0].message.content.strip()
    match = re.search(r'</think>\s*(.*)', conteudo, re.DOTALL)
    return match.group(1).strip() if match else conteudo

In [16]:
resultados = []

for i, url in enumerate(link_questoes['Link'][17:], start=17):
	print(f"{'='*20} Questão {i} {'='*20}")

	questao_texto = Questao(url).texto()

	# Classificação do assunto
	prompt_assunto = PROMPT_ASSUNTO.format(
		enunciado=questao_texto,
		regras_da_saida=REGRAS_SAIDA.format(
			regra="Fundamentos de Matemática, Fundamentos de Computação ou Algoritmos e Estruturas de Dados"
		)
	)
	assunto_final = obter_conteudo_formatado(prompt_assunto)
	print(f"Assunto: {assunto_final}")

	# Classificação do nível de dificuldade
	prompt_dificuldade = PROMPT_DIFICULDADE.format(
		enunciado=questao_texto,
		regras_da_saida=REGRAS_SAIDA.format(regra="Fácil, Médio ou Difícil")
	)
	nivel_final = obter_conteudo_formatado(prompt_dificuldade)
	print(f"Nível de dificuldade: {nivel_final}")

	# Classificação do subassunto
	subassuntos_str = subassuntos(assunto_final)
	prompt_subassunto = PROMPT_SUBASSUNTO.format(
		enunciado=questao_texto,
		conteudo_estruturado_subassuntos=subassuntos_str,
		regras_da_saida=REGRAS_SAIDA.format(regra="<Grupo_subassunto> | <Subassunto> | <Nível>")
	)
	subassunto_final = obter_conteudo_formatado(prompt_subassunto)

	try:
		grupo_subassunto, subassunto, nivel_subassunto = [s.strip() for s in subassunto_final.split('|')]
		print(f"Grupo de subassunto: {grupo_subassunto}")
		print(f"Subassunto: {subassunto}")
		print(f"Nível de subassunto: {nivel_subassunto}")
	except Exception:
		grupo_subassunto, subassunto, nivel_subassunto = '', '', ''

	resultados.append({
		'url': url,
		'assunto': assunto_final,
		'nivel': nivel_final,
		'grupo_subassunto': grupo_subassunto,
		'subassunto': subassunto,
		'nivel_subassunto': nivel_subassunto
	})
	time.sleep(10)
df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv('data/base_questoes_classificadas.csv', index=False)

==================== Questão 17 ====================
Assunto: Algoritmos e Estruturas de Dados
Nível de dificuldade: Médio
Grupo de subassunto: Estratégias de Algoritmos
Subassunto: Estratégias simples de iteração e repetição
Nível de subassunto: PJ
==================== Questão 18 ====================
Assunto: Algoritmos e Estruturas de Dados
Nível de dificuldade: Difícil
Grupo de subassunto: Estratégias de Algoritmos
Subassunto: Programação dinâmica
Nível de subassunto: P1
==================== Questão 19 ====================
Assunto: Algoritmos e Estruturas de Dados
Nível de dificuldade: Médio
Grupo de subassunto: Estratégias de Algoritmos
Subassunto: Backtracking simples (busca exaustiva recursiva)
Nível de subassunto: PJ
==================== Questão 20 ====================
Assunto: Algoritmos e Estruturas de Dados
Nível de dificuldade: Médio
Grupo de subassunto: Estratégias de Algoritmos
Subassunto: Estratégias simples de iteração e repetição
Nível de subassunto: PJ
================

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jwywcrtvfavvbkb6s8h7ynna` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99642, Requested 741. Please try again in 5m30.378999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [18]:
df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv('src/data/base_questoes_classificadas.csv', index=False)